In [2]:
import pandas as pd

df = pd.read_csv('data/raw_data_10k.csv')
df

,_sa_instance_state_question,context,id_question,author_id,upvote,turns,is_answerable_zs,rejected,scope,text_question,...,id_answer,setting,text,_sa_instance_state_rating,id_rating,answer_id,timestamp_rating,author_id_rating,text_rating,value
0,<sqlalchemy.orm.state.InstanceState object at ...,"In an article about '575 pm . However , there ...",38628.0,1.0,0.0,single,False,True,single-paragraph,What type of analyzer did the Surveyor 5 lunar...,...,77255,zs,The Surveyor 5 lunar probe used an Alpha-Parti...,<sqlalchemy.orm.state.InstanceState object at ...,77255,77255,2024-06-26 07:02:41,3,"The answer provided is partially correct, but ...",2
1,<sqlalchemy.orm.state.InstanceState object at ...,"In an article about '575 pm . However , there ...",38628.0,1.0,0.0,single,False,True,single-paragraph,What type of analyzer did the Surveyor 5 lunar...,...,77256,ic,"The Surveyor 5 lunar probe used an ""alpha-scat...",<sqlalchemy.orm.state.InstanceState object at ...,77256,77256,2024-06-26 07:02:45,3,This answer is fully accurate and detailed. It...,5
2,<sqlalchemy.orm.state.InstanceState object at ...,In an article about 'Prince of Persia : The Sa...,6976.0,1.0,0.0,single,False,True,single-paragraph,What did 1UP.com compare the camera control of...,...,13951,zs,"According to 1UP.com, the camera control of Pr...",<sqlalchemy.orm.state.InstanceState object at ...,13951,13951,2024-06-26 04:30:37,3,The provided answer states that 1UP.com compar...,0
3,<sqlalchemy.orm.state.InstanceState object at ...,In an article about 'Prince of Persia : The Sa...,6976.0,1.0,0.0,single,False,True,single-paragraph,What did 1UP.com compare the camera control of...,...,13952,ic,"According to the text, 1UP.com described the c...",<sqlalchemy.orm.state.InstanceState object at ...,13952,13952,2024-06-26 04:30:50,3,"The answer accurately quotes the text, stating...",5
4,<sqlalchemy.orm.state.InstanceState object at ...,"In an article about 'Surrender of Japan', sect...",9543.0,1.0,0.0,single,False,True,single-paragraph,To whom did Kiichiro Hiranuma suggest Emperor ...,...,19085,zs,Kiichiro Hiranuma suggested that Emperor Hiroh...,<sqlalchemy.orm.state.InstanceState object at ...,19085,19085,2024-06-26 04:41:17,3,The answer provided suggests that Kiichiro Hir...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,<sqlalchemy.orm.state.InstanceState object at ...,"In an article about 'Ames Project', section 'U...",39295.0,1.0,0.0,single,True,False,single-paragraph,What process was used to remove iron impuritie...,...,78590,ic,The process used to remove iron impurities fro...,<sqlalchemy.orm.state.InstanceState object at ...,78590,78590,2024-06-26 07:06:09,3,This answer is fully accurate and detailed. It...,5
19996,<sqlalchemy.orm.state.InstanceState object at ...,"In an article about 'La Luz del Mundo', sectio...",20291.0,1.0,0.0,single,True,False,single-paragraph,"As of 2011, what distinction does the Houston,...",...,40581,zs,"As of 2011, the Houston, Texas temple holds th...",<sqlalchemy.orm.state.InstanceState object at ...,40581,40581,2024-06-26 05:33:10,3,This answer is fully accurate and detailed. Th...,5
19997,<sqlalchemy.orm.state.InstanceState object at ...,"In an article about 'La Luz del Mundo', sectio...",20291.0,1.0,0.0,single,True,False,single-paragraph,"As of 2011, what distinction does the Houston,...",...,40582,ic,"As of 2011, the Houston, Texas temple is the l...",<sqlalchemy.orm.state.InstanceState object at ...,40582,40582,2024-06-26 05:33:58,3,This answer is fully accurate and detailed. It...,5
19998,<sqlalchemy.orm.state.InstanceState object at ...,"In an article about 'Jena Six', section 'Backg...",36063.0,1.0,0.0,single,False,True,single-paragraph,What percentage of students at Jena High Schoo...,...,72125,zs,According to the National Center for Education...,<sqlalchemy.orm.state.InstanceState object at ...,72125,72125,2024-06-26 06:51:09,3,The provided answer is significantly incorrect...,1


In [2]:
df.columns

Index(['_sa_instance_state_question', 'context', 'id_question', 'author_id',
       'upvote', 'turns', 'is_answerable_zs', 'rejected', 'scope',
       'text_question', 'paragraph_id', 'timestamp', 'downvote', 'filtered',
       'is_answerable_ic', 'processed_question', 'is_rejected',
       '_sa_instance_state_paragraph', 'subsection_name', 'id_paragraph',
       'text_paragraph', 'text_cleaned', 'is_bad', 'processed_paragraph',
       'subsubsection_name', 'page_name', 'section_name', 'section_hierarchy',
       'word_count', 'within_page_order', 'original_entry_id',
       '_sa_instance_state', 'hash', 'model', 'prompt', 'id', 'username',
       '_sa_instance_state_answer', 'author_id_answer', 'timestamp_answer',
       'question_id', 'processed', 'id_answer', 'setting', 'text',
       '_sa_instance_state_rating', 'id_rating', 'answer_id',
       'timestamp_rating', 'author_id_rating', 'text_rating', 'value'],
      dtype='object')

In [4]:
from vllm import SamplingParams

def generate_answers(questions: list[str], llm):
    answers = []
    prompts = []

    for question in questions:
        # Process prompt template
        prompt_template = "{PROMPT_PREFIX}{CONTEXT_PROMPT}Answer the following question in a succinct manner: {QUESTION}\n{PROMPT_SUFFIX}"

        prompt = prompt_template.format(
            CONTEXT_PROMPT="",
            QUESTION=question.text,
            PROMPT_PREFIX="",
            PROMPT_SUFFIX="",
        )
        sampling_params = SamplingParams(max_tokens=200, temperature=0)

        prompts.append(prompt)

    batch_prompts = [p[0] for p in prompts]
    outputs = llm.generate(batch_prompts, sampling_params)

    for i, output in enumerate(outputs):
        answer_text = output.outputs[0].text.strip()
        if answer_text:
            answers.append(answer_text)
    return answers

In [ ]:
from tqdm.notebook import tqdm
from askmevllm.dataset.ratings import generate_answer_ratings
from vllm import LLM
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
llm = LLM("meta-llama/Meta-Llama-3-70B-Instruct", tensor_parallel_size=2, seed=42)


In [7]:
import pandas as pd

def prepare_google_sheet(df, sample_size=100):
    # Sample 100 questions
    questions_sample = df.sample(n=sample_size)

    # Create zeroshot and in context entries
    zeroshot_entries = questions_sample.copy()
    zeroshot_entries['type'] = 'zeroshot'
    zeroshot_entries['is_answerable_zs'] = ''
    zeroshot_entries['is_answerable_ic'] = ''
    zeroshot_entries['answer_id'] = None
    zeroshot_entries['answer'] = None
    zeroshot_entries['rating'] = None

    in_context_entries = questions_sample.copy()
    in_context_entries['type'] = 'in context'
    in_context_entries['is_answerable_zs'] = ''
    in_context_entries['is_answerable_ic'] = ''
    in_context_entries['rating'] = ''

    # Select only necessary columns
    zeroshot_entries = zeroshot_entries[[
        'index', 'id_question', 'text_question', 'context', 'type', 
        'is_answerable_zs', 'is_answerable_ic'
    ]]
    in_context_entries = in_context_entries[[
        'index', 'id_question', 'text_question', 'context', 'type', 
        'id_answer', 'text', 'rating', 'is_answerable_zs', 'is_answerable_ic'
    ]]

    # Rename columns for ease of use
    zeroshot_entries.columns = [
        'index', 'question_id', 'question', 'context', 'type', 
        'is_answerable_zs', 'is_answerable_ic'
    ]
    in_context_entries.columns = [
        'index', 'question_id', 'question', 'context', 'type', 
        'answer_id', 'answer', 'rating', 'is_answerable_zs', 'is_answerable_ic'
    ]

    # Concatenate zeroshot and in context entries
    evaluation_sheet = pd.concat([zeroshot_entries, in_context_entries], ignore_index=True)

    return evaluation_sheet

# Example usage
# df = pd.read_csv('prepared_data.csv')  # Load your prepared DataFrame
df['index'] = df.index  # Add an index column for easy data cleaning
evaluation_sheet = prepare_google_sheet(df, sample_size=100)
evaluation_sheet.to_csv('data/human_in_the_loop_evaluation_sheet.csv', index=False)
